In [1]:
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Flatten, Dropout
from keras import backend 
from keras import optimizers 

#loader = load_img('./train/happy/MEL_dia99_utt13_positive_HAPjpg.png')
#print(img_to_array(loader).shape)

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.listdir('./drive/My Drive/meldnew/train')


['happy', 'neutral', 'fear', 'sad', 'disgust']

In [0]:
path_train = './drive/My Drive/meldnew/train'

In [0]:
path_val = './drive/My Drive/meldnew/validation'

In [8]:
img_wid = 300
img_height= 300


if backend.image_data_format() == 'channels_first':
    input_shape = (3, img_wid, img_height)
else:
    input_shape=(img_wid, img_height, 3)

datagen = ImageDataGenerator(rescale= 1/255, width_shift_range=1.0, zoom_range=0.2)
train = datagen.flow_from_directory(path_train, class_mode='categorical', batch_size=30, target_size = (img_wid, img_height) )
print(train)

datagen_val = ImageDataGenerator(rescale= 1/255)


val =datagen_val.flow_from_directory(path_val, class_mode='categorical', batch_size=30, target_size = (img_wid, img_height) )


model_cnn = Sequential()
model_cnn.add(Conv2D(filters=32, kernel_size=(2,2),padding='same',strides=(1,1), input_shape=input_shape))
model_cnn.add(Activation('relu'))
model_cnn.add(MaxPooling2D(pool_size= (2,2), strides=2))


model_cnn.add(Conv2D(filters= 64, kernel_size=(2,2), padding='valid', strides=(1,1)))
model_cnn.add(Activation('relu'))
model_cnn.add(MaxPooling2D(pool_size=(2,2), strides=2))

model_cnn.add(Flatten())
model_cnn.add(Dense(64))
model_cnn.add(Activation('relu'))

model_cnn.add(Dropout(0.25))

model_cnn.add(Dense(5))
model_cnn.add(Activation('sigmoid'))

#rms = optimizers.RMSprop(learning_rate=0.1)

model_cnn.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model_cnn.fit_generator(train, epochs=5, steps_per_epoch= 7800//78 )

Found 7372 images belonging to 5 classes.
Found 830 images belonging to 5 classes.
Epoch 1/5
100/100 [==============================] - 312s 3s/step - loss: 1.5682 - acc: 0.5200
Epoch 2/5
100/100 [==============================] - 304s 3s/step - loss: 1.1385 - acc: 0.6298
Epoch 3/5
100/100 [==============================] - 304s 3s/step - loss: 1.1056 - acc: 0.6333
Epoch 4/5
100/100 [==============================] - 303s 3s/step - loss: 1.1379 - acc: 0.6093
Epoch 5/5
100/100 [==============================] - 301s 3s/step - loss: 1.0826 - acc: 0.6295


In [10]:
model_cnn.evaluate_generator(val)

[1.0738943700330803, 0.6228915750980377]

In [0]:
# Saving the model to google drive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
model_cnn.save('model.json')
model_file = drive.CreateFile({'title' : 'model.json'})
model_file.SetContentFile('model.json')
model_file.Upload()

In [15]:
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1GvSa6Prxn8ysr4QN1eF85XL1WseFDgTW'})

In [0]:
#saving Weights
model_cnn.save_weights('weights.h5')
weights_file = drive.CreateFile({'title' : 'weights.h5'})
weights_file.SetContentFile('weights.h5')
weights_file.Upload()


In [17]:
drive.CreateFile({'id': model_file.get('id')})

GoogleDriveFile({'id': '1GvSa6Prxn8ysr4QN1eF85XL1WseFDgTW'})